In [ ]:
from bs4 import BeautifulSoup
import requests
import lxml
import time
import json
import os
from os import path

In [ ]:
teamDir = "247\\sec\\teams\\"
recruitDir = "247\\sec\\recruits\\"

In [ ]:
all_recruits = []

for file in os.listdir(teamDir):
    gameSoup = BeautifulSoup(open(teamDir + file, "r").read(), 'lxml')
    starter = "ri-page__list-item"
    team = file.split('_')[0]
    y = (file.split('_')[1]).split('.')[0]
    for x in gameSoup.find_all("li", class_="ri-page__list-item"):
        #append player to sqlinsert list
        player={}
        player_status = ""
        if (x.find("p", class_="commit-date") is not None):
            player_status = x.find("p", class_="commit-date").text
        if (player_status.strip() == 'Enrolled' or player_status.strip() == 'Signed'):
            player['school'] = team
            player['year'] = y
            #Name
            if (x.find("a", class_="ri-page__name-link") is not None):
                player['playerName'] = x.find("a", class_="ri-page__name-link").text
            #School/CityState
            if (x.find("span", class_="meta") is not None):
                locationInfoRaw = x.find("span", class_="meta")
                locationInfoList = locationInfoRaw.text.split("(")
                player['highSchool'] = (locationInfoList[0].strip())
                cityState = locationInfoList[1].split(', ')
                if (len(cityState) > 1):
                    #print(cityState)
                    state = cityState[1].strip()
                    #print(state.rstrip(')'))
                    player['city'] = (cityState[0].strip())
                    player['state'] = (state.rstrip(')'))
                else:
                    player['city'] = "None"
                    player['state'] = "None"
            #Composite Rating
            if (x.find("span", class_="score") is not None):
                player['compRating'] = (x.find("span", class_="score").text)
            #Position
            if (x.find("div", class_="position") is not None):
                player['position'] = ((x.find("div", class_="position").text).strip())
            #Height/Weight
            if (x.find("div", class_="metrics") is not None):
                heightWeight = x.find("div", class_="metrics").text.strip()
                height = (heightWeight.split(' / '))[0]
                inchHeightPre = height.split('-')
                if (inchHeightPre[0] != ''):
                    inchHeight = int(inchHeightPre[0])*12 + float(inchHeightPre[1])
                else:
                    inchHeight = '0.0'
                weight = (heightWeight.split(' / '))[1]
                if (weight == '-'):
                    weight = '0'
                player['height'] = (inchHeight)
                player['weight'] = (weight)
            #Stars
            ratingChildren = x.find_all("span", class_='icon-starsolid yellow')
            i = 0
            for child in ratingChildren:
                i = i + 1
            player['compStars'] = (i)
            #National Rank
            if (x.find("a", class_="natrank") is not None):
                player['nationalRank'] = ((x.find("a", class_="natrank").text).strip())
            #Position Rank
            if (x.find("a", class_="posrank") is not None):
                player['positionRank'] = ((x.find("a", class_="posrank").text).strip())
            #State Rank
            if (x.find("a", class_="sttrank") is not None):
                player['stateRank'] = ((x.find("a", class_="sttrank").text).strip())
        if player:
            all_recruits.append(player)  
print(all_recruits)

In [ ]:
with open("scraped_json\\247_sec.json", "w") as write_file:
    json.dump(all_recruits, write_file)